# Tests

In [3]:
import os
import sys
import logging
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH

nq = Nanuq()
pho = Phenotips()
bssh = BSSH()

In [4]:
class Args:
    def __init__(self, run='LH00336_0006', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'A00516_0445', 'level': 'debug'}

In [ ]:
foo = None
if foo: 
    print('Foo is not None')
else:
    print("Foo is None")